In [4]:
%AddDeps org.nd4j nd4j-native-platform 1.0.0-beta --transitive 
%AddDeps org.deeplearning4j deeplearning4j-core 1.0.0-beta --transitive 
%AddDeps org.datavec datavec-spark_2.11 1.0.0-beta_spark_2 --transitive 

Marking org.nd4j:nd4j-native-platform:1.0.0-beta for download
Obtained 73 files
Marking org.deeplearning4j:deeplearning4j-core:1.0.0-beta for download
Obtained 171 files
Marking org.datavec:datavec-spark_2.11:1.0.0-beta_spark_2 for download
Obtained 60 files


In [5]:
import org.deeplearning4j.nn.multilayer.MultiLayerNetwork
import org.deeplearning4j.nn.conf.MultiLayerConfiguration
import org.deeplearning4j.nn.conf.NeuralNetConfiguration
import org.deeplearning4j.nn.weights.WeightInit
import org.nd4j.linalg.learning.config.Adam
import org.deeplearning4j.nn.conf.layers.LSTM
import org.nd4j.linalg.activations.Activation
import org.nd4j.linalg.lossfunctions.LossFunctions
import org.deeplearning4j.nn.conf.layers.RnnOutputLayer
def createModel(): MultiLayerNetwork = {
    val conf: MultiLayerConfiguration = new  NeuralNetConfiguration.Builder()
    .seed(123456)
    .optimizationAlgo(org.deeplearning4j.nn.api.OptimizationAlgorithm.STOCHASTIC_GRADIENT_DESCENT)
    //.updater(new Adam(2e-3))
    .l2(1e-5)
    .weightInit(WeightInit.XAVIER)
    .list()
    .layer(0, new LSTM.Builder().name("encoder0").nIn(784).nOut(800).build())
    .layer(1, new LSTM.Builder().name("encoder1").nIn(800).nOut(250).build())
    .layer(2, new LSTM.Builder().name("encoder2").nIn(250).nOut(10).build())
    .layer(3, new LSTM.Builder().name("encoder3").nIn(10).nOut(250).build())
    .layer(4, new LSTM.Builder().name("encoder4").nIn(250).nOut(800).build())
    .layer(5, new RnnOutputLayer.Builder().name("output").nIn(800).nOut(784)
           .activation(Activation.IDENTITY).lossFunction(LossFunctions.LossFunction.MSE).build()
          )
    .pretrain(false).backprop(true)
    .build()
    val net:MultiLayerNetwork = new MultiLayerNetwork(conf)
    net.init()
    net
}

createModel: ()org.deeplearning4j.nn.multilayer.MultiLayerNetwork


In [4]:
def createModel2(): MultiLayerNetwork = {
    val conf: MultiLayerConfiguration = new  NeuralNetConfiguration.Builder()
    .seed(123456)
    .optimizationAlgo(org.deeplearning4j.nn.api.OptimizationAlgorithm.STOCHASTIC_GRADIENT_DESCENT)
    //.updater(new Adam(2e-3))
    .l2(1e-5)
    .weightInit(WeightInit.XAVIER)
    .list()
    .layer(0, new LSTM.Builder().name("encoder0").nIn(28).nOut(30).build())
    .layer(1, new LSTM.Builder().name("encoder1").nIn(30).nOut(10).build())
    .layer(2, new LSTM.Builder().name("encoder2").nIn(10).nOut(30).build())
    .layer(3, new RnnOutputLayer.Builder().name("output").nIn(30).nOut(28)
           .activation(Activation.IDENTITY).lossFunction(LossFunctions.LossFunction.MSE).build()
          )
    .pretrain(false).backprop(true)
    .build()
    val net:MultiLayerNetwork = new MultiLayerNetwork(conf)
    net.init()
    net
}

createModel2: ()org.deeplearning4j.nn.multilayer.MultiLayerNetwork


In [5]:
import org.deeplearning4j.ui.api.Utils
import org.nd4j.linalg.dataset.api.iterator.DataSetIterator
//import org.deeplearning4j.examples.unsupervised.anomalydetection.MnistIterator
//import org.deeplearning4j.examples.unsupervised.anomalydetection.MnistLoader
//import org.deeplearning4j.examples.unsupervised.anomalydetection.MnistIterator
import org.nd4j.linalg.api.ndarray.INDArray
import org.deeplearning4j.datasets.iterator.impl.MnistDataSetIterator
import java.util.Random
import java.util.ArrayList

In [6]:
val featuresTrain = new ArrayList[INDArray]
val featuresTest = new ArrayList[INDArray]
val labelsTest = new ArrayList[INDArray]
val rand = new Random(12345)

featuresTrain = []
featuresTest = []
labelsTest = []
rand = java.util.Random@590a82df


java.util.Random@590a82df

In [7]:
/* main function */
val net = createModel2()
net.setListeners(new org.deeplearning4j.optimize.listeners.ScoreIterationListener(10))
val iter = new MnistDataSetIterator(100,5000,false)

net = org.deeplearning4j.nn.multilayer.MultiLayerNetwork@2fdc6b8
iter = org.deeplearning4j.datasets.iterator.impl.MnistDataSetIterator@1e395003


org.deeplearning4j.datasets.iterator.impl.MnistDataSetIterator@1e395003

In [8]:
while (iter.hasNext()) {
    val ds = iter.next;
    val split = ds.splitTestAndTrain(80, rand)
    featuresTrain.add(split.getTrain.getFeatures().reshape(80, 28, 28))
    featuresTest.add(split.getTest.getFeatures.reshape(20, 28, 28))
    labelsTest.add(split.getTest.getLabels.argMax(1))
}

In [9]:
val nEpochs = 1
(1 to nEpochs).foreach{ epoch =>  
    for(data <- featuresTrain.toArray){
        net.fit(data.asInstanceOf[INDArray],data.asInstanceOf[INDArray]);
    }
    println("Epoch " + epoch + " complete");
}

Epoch 1 complete


nEpochs = 1


1

In [29]:
import scala.collection.mutable.ListBuffer
val evalList = ListBuffer[(Double, INDArray)]()
var totalScore:Double = 0

evalList = ListBuffer()
totalScore = 0.0


lastException: Throwable = null


0.0

In [30]:
import org.nd4j.linalg.dataset.DataSet 

(0 until featuresTest.size).foreach( i => {
    val testData = featuresTest.get(i)
    val nRows = testData.shape()(0).asInstanceOf[Int]
    (1 until nRows).foreach( j => {
        val example = testData.getRow(j).reshape(1, 28, 28)
        val score = net.score(new DataSet(example, example))
        totalScore = totalScore + score
        evalList.append((score, example))
    })
    println("featuresTest " + i + " complete")
})

featuresTest 0 complete
featuresTest 1 complete
featuresTest 2 complete
featuresTest 3 complete
featuresTest 4 complete
featuresTest 5 complete
featuresTest 6 complete
featuresTest 7 complete
featuresTest 8 complete
featuresTest 9 complete
featuresTest 10 complete
featuresTest 11 complete
featuresTest 12 complete
featuresTest 13 complete
featuresTest 14 complete
featuresTest 15 complete
featuresTest 16 complete
featuresTest 17 complete
featuresTest 18 complete
featuresTest 19 complete
featuresTest 20 complete
featuresTest 21 complete
featuresTest 22 complete
featuresTest 23 complete
featuresTest 24 complete
featuresTest 25 complete
featuresTest 26 complete
featuresTest 27 complete
featuresTest 28 complete
featuresTest 29 complete
featuresTest 30 complete
featuresTest 31 complete
featuresTest 32 complete
featuresTest 33 complete
featuresTest 34 complete
featuresTest 35 complete
featuresTest 36 complete
featuresTest 37 complete
featuresTest 38 complete
featuresTest 39 complete
featuresTe

In [31]:
evalList.sortWith(_._1 > _._1).foreach( a => println(a._1))

9.575557281409175
9.56095897993038
9.55355955714857
9.422991189054263
8.967417698366484
8.66649584952441
8.642566798533895
8.622287322913081
8.52716512181096
8.522840345025246
8.342645490288918
8.157508967723349
8.022530264633225
7.881628971559844
7.770500300731161
7.72099097979087
7.627325993044219
7.596766998206049
7.591119611382667
7.5536979761636385
7.518264343176752
7.5156327470202235
7.509284000856719
7.4926579153028685
7.4758443646262505
7.4250342319183815
7.1947148000684935
7.185751896364532
7.175168836236183
7.100979513900803
7.082206843699911
7.0365743450950005
7.012079084038917
6.976304308125134
6.954256447843326
6.93123747462904
6.924028241753761
6.913362893155826
6.907834851861183
6.907034855348907
6.903160757797288
6.896516645074073
6.849412218281384
6.837759953005157
6.833392669592768
6.807176435113136
6.803862553102813
6.780754206981161
6.77216023899846
6.733081390295893
6.715458033885458
6.711753417883783
6.708064605627924
6.683463078005157
6.672784105488415
6.65487519

In [33]:
evalList.sortWith((a, b) => a._1 > b._1).foreach( a => println(a._1))

9.575557281409175
9.56095897993038
9.55355955714857
9.422991189054263
8.967417698366484
8.66649584952441
8.642566798533895
8.622287322913081
8.52716512181096
8.522840345025246
8.342645490288918
8.157508967723349
8.022530264633225
7.881628971559844
7.770500300731161
7.72099097979087
7.627325993044219
7.596766998206049
7.591119611382667
7.5536979761636385
7.518264343176752
7.5156327470202235
7.509284000856719
7.4926579153028685
7.4758443646262505
7.4250342319183815
7.1947148000684935
7.185751896364532
7.175168836236183
7.100979513900803
7.082206843699911
7.0365743450950005
7.012079084038917
6.976304308125134
6.954256447843326
6.93123747462904
6.924028241753761
6.913362893155826
6.907834851861183
6.907034855348907
6.903160757797288
6.896516645074073
6.849412218281384
6.837759953005157
6.833392669592768
6.807176435113136
6.803862553102813
6.780754206981161
6.77216023899846
6.733081390295893
6.715458033885458
6.711753417883783
6.708064605627924
6.683463078005157
6.672784105488415
6.65487519

0
1
2
3
4
5
6
7
8
9
10
